In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# TensorFlow and Keras Imports
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models, metrics
from tensorflow.keras.utils import Sequence
from tensorflow.keras.callbacks import ModelCheckpoint

# Image Processing Libraries
import cv2
from cv2 import imread,resize
from scipy.ndimage import label, find_objects

# Data Handling Libraries
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import random

# Visualization Libraries
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

# File and Operating System Libraries
import os

# Warnings Management
import warnings
warnings.filterwarnings('ignore')

# GPU Configuration
# os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
print(tf.config.list_physical_devices('GPU'))

#DataGeneratrion
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# PATHS
IMG_PATH = '/kaggle/input/chest-x-ray-lungs-segmentation/Chest-X-Ray/Chest-X-Ray/image'
MSK_PATH = '/kaggle/input/chest-x-ray-lungs-segmentation/Chest-X-Ray/Chest-X-Ray/mask'

In [ ]:
print(tf.test.is_gpu_available())
print(tf.test.is_built_with_cuda())

In [ ]:
sample_img = cv2.imread('/kaggle/input/chest-x-ray-lungs-segmentation/Chest-X-Ray/Chest-X-Ray/image/1000.png',0)
plt.imshow(sample_img)

In [ ]:
sample_img.shape

In [ ]:
sample_msk = cv2.imread('/kaggle/input/chest-x-ray-lungs-segmentation/Chest-X-Ray/Chest-X-Ray/mask/1000.png',0)
plt.imshow(sample_msk)

In [ ]:
sample_msk.shape

Data Generation and processing

In [ ]:
img_path = '/kaggle/input/chest-x-ray-lungs-segmentation/Chest-X-Ray/Chest-X-Ray/image'
msk_path = '/kaggle/input/chest-x-ray-lungs-segmentation/Chest-X-Ray/Chest-X-Ray/mask'

In [ ]:
img_files, msk_files = sorted(os.listdir(img_path)),sorted(os.listdir(msk_path))

train_img_files,val_img_files,train_msk_files,val_msk_files = train_test_split(img_files,msk_files,test_size=0.2,random_state=1)

In [ ]:
img_datagen = ImageDataGenerator(
    # rotation_range=90,
    width_shift_range=0.05,
    height_shift_range=0.05,
    # horizontal_flip=True,
    # vertical_flip=True,
    # shear_range=0.3,
    # zoom_range=0.5,
    fill_mode='reflect',
    # rescale = 1./255.
)

msk_datagen = ImageDataGenerator(
    # rotation_range=90,
    width_shift_range=0.05,
    height_shift_range=0.05,
    # horizontal_flip=True,
    # vertical_flip=True,
    # shear_range=0.3,
    # zoom_range=0.5,
    fill_mode='reflect',
    # rescale = 1./255.
)

batch_size = 16
target_size = (256,256)

def image_mask_generator(image_files,mask_files,batch_size):
    while True:

        combined = list(zip(image_files, mask_files))
        random.shuffle(combined)
        image_files, mask_files = zip(*combined)
        
        for i in range(0,len(image_files),batch_size):
            batch_img_files = image_files[i:i+batch_size]
            batch_msk_files = mask_files[i:i+batch_size]

            msk_batch = []
            img_batch = []

            for img_file, msk_file in zip(batch_img_files,batch_msk_files):
                img = cv2.imread(os.path.join(img_path,img_file), cv2.IMREAD_COLOR)
                img = cv2.resize(img,target_size)/255.
                # img = np.expand_dims(img, axis = -1)

                msk = cv2.imread(os.path.join(msk_path,msk_file), cv2.IMREAD_GRAYSCALE)
                msk = cv2.resize(msk,target_size)/255.
                msk = (msk>0).astype(np.uint8)
                msk = np.expand_dims(msk, axis = -1)

                img_batch.append(img)
                msk_batch.append(msk)

            img_batch = np.array(img_batch)
            msk_batch = np.array(msk_batch)

            # seed = np.random.randint(1,1000)
            # img_batch = next(img_datagen.flow(img_batch, batch_size = batch_size, seed=seed, shuffle = False))
            # msk_batch = next(msk_datagen.flow(msk_batch, batch_size = batch_size, seed=seed, shuffle = False))

            yield img_batch, msk_batch

In [ ]:
import numpy as np
import cv2
import os
import matplotlib.pyplot as plt

# Function to visualize the original and augmented image-mask pairs
def visualize_generator(generator, image_files, mask_files, num_samples=3):
    """
    Plots original images and masks alongside augmented versions.
    """
    # Load a batch of images & masks from generator
    img_batch, msk_batch = next(generator)

    # Show the first `num_samples` images and masks
    for i in range(num_samples):
        img = cv2.imread(os.path.join(img_path, image_files[i]), cv2.IMREAD_COLOR)
        img = cv2.resize(img, target_size) / 255.0  # Normalize

        msk = cv2.imread(os.path.join(msk_path, mask_files[i]), cv2.IMREAD_GRAYSCALE)
        msk = cv2.resize(msk, target_size) / 255.0  # Normalize
        msk = (msk > 0).astype(np.uint8)  # Binarize mask

        aug_img = img_batch[i]
        aug_msk = msk_batch[i]

        plt.figure(figsize=(8, 4))

        # Original Image
        plt.subplot(2, 2, 1)
        plt.imshow(img)
        plt.title("Original Image")
        plt.axis("off")

        # Original Mask
        plt.subplot(2, 2, 2)
        plt.imshow(msk, cmap="gray")
        plt.title("Original Mask")
        plt.axis("off")

        # Augmented Image
        plt.subplot(2, 2, 3)
        plt.imshow(aug_img)
        plt.title("Augmented Image")
        plt.axis("off")

        # Augmented Mask
        plt.subplot(2, 2, 4)
        plt.imshow(aug_msk.squeeze(), cmap="gray")
        plt.title("Augmented Mask")
        plt.axis("off")

        plt.tight_layout()
        plt.show()

# Create the generator
test_generator = image_mask_generator(train_img_files, train_msk_files, batch_size=8)

# Visualize results
visualize_generator(test_generator, train_img_files, train_msk_files)


In [ ]:
train_generator = image_mask_generator(train_img_files, train_msk_files, batch_size)
valid_generator = image_mask_generator(val_img_files, val_msk_files, batch_size)

In [ ]:
def unet(input_shape, num_classes):
    # Input layer for the model
    inputs = layers.Input(shape=input_shape)
    
    # Contracting Path (Encoder)
    # First block with 64 filters
    conv1 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    conv1 = layers.BatchNormalization()(conv1)  # Add Batch Normalization
    conv1 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(conv1)
    conv1 = layers.BatchNormalization()(conv1)  # Add Batch Normalization
    pool1 = layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(conv1)  # Downsample

    # Second block with 128 filters
    conv2 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(pool1)
    conv2 = layers.BatchNormalization()(conv2)  # Add Batch Normalization
    conv2 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(conv2)
    conv2 = layers.BatchNormalization()(conv2)  # Add Batch Normalization
    pool2 = layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(conv2)  # Downsample

    # Third block with 256 filters
    conv3 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(pool2)
    conv3 = layers.BatchNormalization()(conv3)  # Add Batch Normalization
    conv3 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(conv3)
    conv3 = layers.BatchNormalization()(conv3)  # Add Batch Normalization
    pool3 = layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(conv3)  # Downsample

    # Bottleneck layer with 512 filters
    conv4 = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(pool3)
    conv4 = layers.BatchNormalization()(conv4)  # Add Batch Normalization
    conv4 = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(conv4)
    conv4 = layers.BatchNormalization()(conv4)  # Add Batch Normalization

    # Expansive Path (Decoder)
    # First upsampling block
    up6 = layers.Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(conv4)  # Upsample
    merge6 = layers.concatenate([up6, conv3])  # Concatenate with the corresponding encoder block
    conv6 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(merge6)
    conv6 = layers.BatchNormalization()(conv6)  # Add Batch Normalization
    conv6 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(conv6)
    conv6 = layers.BatchNormalization()(conv6)  # Add Batch Normalization

    # Second upsampling block
    up7 = layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(conv6)  # Upsample
    merge7 = layers.concatenate([up7, conv2])  # Concatenate with the corresponding encoder block
    conv7 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(merge7)
    conv7 = layers.BatchNormalization()(conv7)  # Add Batch Normalization
    conv7 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(conv7)
    conv7 = layers.BatchNormalization()(conv7)  # Add Batch Normalization

    # Third upsampling block
    up8 = layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(conv7)  # Upsample
    merge8 = layers.concatenate([up8, conv1])  # Concatenate with the corresponding encoder block
    conv8 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(merge8)
    conv8 = layers.BatchNormalization()(conv8)  # Add Batch Normalization
    conv8 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(conv8)
    conv8 = layers.BatchNormalization()(conv8)  # Add Batch Normalization

    # Output layer for segmentation
    outputs = layers.Conv2D(num_classes, (1, 1), activation='sigmoid')(conv8)

    # Create the model
    model = models.Model(inputs=[inputs], outputs=[outputs])
    return model

In [ ]:
# Jaccard Index Metric
def jaccard_index(y_true, y_pred, smooth=100):
    """Calculates the Jaccard index (IoU), useful for evaluating the model's performance."""
    y_true_f = tf.reshape(tf.cast(y_true, tf.float32), [-1])  # Flatten and cast ground truth
    y_pred_f = tf.reshape(tf.cast(y_pred, tf.float32), [-1])  # Flatten and cast predictions
    intersection = tf.reduce_sum(y_true_f * y_pred_f)  # Compute intersection
    total = tf.reduce_sum(y_true_f) + tf.reduce_sum(y_pred_f) - intersection  # Total pixels
    return (intersection + smooth) / (total + smooth)


def dice_coefficient(y_true, y_pred, smooth=1):
    # Flatten and cast true and predicted masks to float32
    y_true_f = tf.reshape(tf.cast(y_true, tf.float32), [-1])  # Flatten and cast y_true to float32
    y_pred_f = tf.reshape(tf.cast(y_pred, tf.float32), [-1])  # Flatten and cast y_pred to float32
    
    # Calculate the intersection between the true and predicted masks
    intersection = tf.reduce_sum(y_true_f * y_pred_f)
    
    # Calculate the Dice coefficient using the formula
    return (2. * intersection + smooth) / (tf.reduce_sum(y_true_f) + tf.reduce_sum(y_pred_f) + smooth)

def dice_loss(y_true, y_pred):
    smooth = 1.0
    y_true_f = tf.reshape(tf.cast(y_true, tf.float32), [-1])
    y_pred_f = tf.reshape(tf.cast(y_pred, tf.float32), [-1])
    intersection = tf.reduce_sum(y_true_f * y_pred_f)
    return 1 - (2. * intersection + smooth) / (tf.reduce_sum(y_true_f) + tf.reduce_sum(y_pred_f) + smooth)


# Using the metrics in model compilation

input_shape = (256,256,3)

strategy = tf.distribute.MirroredStrategy()

with strategy.scope():  # Use GPUs for training
    model = unet(input_shape, 1)
    model.compile(optimizer='adam', 
                  loss='binary_crossentropy', 
                  metrics=['accuracy',dice_coefficient,jaccard_index])

In [ ]:
## ModelCheckpoint: Save the model with the best validation loss during training
checkpoint = ModelCheckpoint(
    'best_model.keras',            # Path to save the model
    monitor='val_dice_coefficient',         # Metric to monitor 
    verbose=1,                  # Print messages when saving the model
    save_best_only=True,        # Save only the best model (with highest metric)
    mode='max',                 # 'max' means the model with the highest metric score will be saved
    save_weights_only=False,     # Save the entire model (not just weights)
)


history = model.fit(
    train_generator,                  # The training data generator or dataset
    validation_data=valid_generator,
    steps_per_epoch=len(train_img_files) // batch_size,
    validation_steps=len(val_img_files) // batch_size,
    epochs=150,              # Number of epochs
    callbacks=[checkpoint],# Callbacks for checkpoint and early stopping
)

In [ ]:
plt.figure(figsize=(16,16))
plt.subplot(221)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')

plt.subplot(222)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.ylabel('Metric Value')
plt.xlabel('Epoch')

plt.subplot(223)
plt.plot(history.history['dice_coefficient'], label='Train Dice Coefficient')
plt.plot(history.history['val_dice_coefficient'], label='Validation Dice Coefficient')
plt.title('Model Dice Coefficient')
plt.ylabel('Metric Value')
plt.xlabel('Epoch')

plt.subplot(224)
plt.plot(history.history['jaccard_index'], label='Train Jaccard Index')
plt.plot(history.history['val_jaccard_index'], label='Validation Jaccard Index')
plt.title('Model Jaccard Index')
plt.ylabel('Metric Value')
plt.xlabel('Epoch')

plt.tight_layout()
plt.show()

In [ ]:
import numpy as np
import cv2
import os
import tensorflow as tf
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score, roc_curve
import matplotlib.pyplot as plt
import timm  # For Vision Transformer
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import pandas as pd
import random

# Custom Metrics
def jaccard_index(y_true, y_pred, smooth=100):
    y_true_f = tf.reshape(tf.cast(y_true, tf.float32), [-1])
    y_pred_f = tf.reshape(tf.cast(y_pred, tf.float32), [-1])
    intersection = tf.reduce_sum(y_true_f * y_pred_f)
    total = tf.reduce_sum(y_true_f) + tf.reduce_sum(y_pred_f) - intersection
    return (intersection + smooth) / (total + smooth)

def dice_coefficient(y_true, y_pred, smooth=1):
    y_true_f = tf.reshape(tf.cast(y_true, tf.float32), [-1])
    y_pred_f = tf.reshape(tf.cast(y_pred, tf.float32), [-1])
    intersection = tf.reduce_sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (tf.reduce_sum(y_true_f) + tf.reduce_sum(y_pred_f) + smooth)

# Load the segmentation model (change the path as needed)
segment_model = load_model('/kaggle/input/c-fvgfb-ghhjfjjhfjh/keras/default/1/model.keras',
                            custom_objects={'dice_coefficient': dice_coefficient, 'jaccard_index': jaccard_index})

# Load data from CSV
data_df = pd.read_csv('/kaggle/input/tbx11k-simplified')
active_tb = data_df[data_df['tb_type']=='active_tb']['fname'].tolist()
active_tb = list(pd.unique(active_tb))
normal = data_df[data_df['image_type']=='healthy']['fname'].tolist()
img_path = '/kaggle/input/tbx11k-simplified/tbx11k-simplified/images'

img_list = []
labels = []
selected_normal = random.sample(normal, len(active_tb))
print(f"shape of selected_normal: {len(selected_normal)}")
print(f"shape of active_tb: {len(active_tb)}")

# Process images for each class
for filename in selected_normal:
    img = cv2.imread(os.path.join(img_path, filename))
    img = cv2.resize(img, (256, 256)) / 255.0  # Normalize to [0,1]
    img_list.append(img)
    labels.append(0)
for filename in active_tb:
    img = cv2.imread(os.path.join(img_path, filename))
    img = cv2.resize(img, (256, 256)) / 255.0
    img_list.append(img)
    labels.append(1)

img_list = np.array(img_list)
labels = np.array(labels)

# Shuffle and split
shuffle_indices = np.random.permutation(len(img_list))
img_list = img_list[shuffle_indices]
labels = labels[shuffle_indices]
X_train, X_val, y_train, y_val = train_test_split(img_list, labels, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_val, y_val, test_size=0.2, random_state=42)

##############################
# Segmentation Preprocessing
##############################

def segment_and_preprocess(img):
    # Convert normalized image to uint8
    img_uint8 = (img * 255).astype(np.uint8)
    # Convert BGR to grayscale
    img_gray = cv2.cvtColor(img_uint8, cv2.COLOR_BGR2GRAY)
    # Resize grayscale image to (48, 40)
    img_resized = cv2.resize(img_gray, (48, 40))
    # Expand dims to get shape (1, 40, 48, 1)
    img_resized = np.expand_dims(np.expand_dims(img_resized, axis=-1), axis=0)
    # Flatten to match segmentation model input
    img_resized_flattened = img_resized.flatten().reshape(1, 1920)
    
    # Create dummy input for the model's second input (shape: (1, 34))
    dummy_input = np.zeros((1, 34))
    
    try:
        # Predict segmentation mask (raw_mask is (8485,))
        raw_mask = segment_model.predict([img_resized_flattened, dummy_input], verbose=0)[0]
        print("Raw mask shape:", raw_mask.shape)  # Should print (8485,)
        
        # Resize this flat mask to 1920 elements
        new_length = 40 * 48  # 1920 elements desired
        x_old = np.linspace(0, 1, raw_mask.shape[0], endpoint=False)
        x_new = np.linspace(0, 1, new_length, endpoint=False)
        mask_resized_flat = np.interp(x_new, x_old, raw_mask)
        
        # Threshold the interpolated mask
        mask_thresholded = (mask_resized_flat > 0.5).astype(np.float32)
        # Reshape to (40, 48)
        mask_reshaped = mask_thresholded.reshape(40, 48)
        print("Reshaped mask shape:", mask_reshaped.shape)  # Should print (40, 48)
        return mask_reshaped
        
    except ValueError as e:
        print(f"Error during prediction: {e}")
        return None

# Apply segmentation preprocessing
X_train_processed = np.array([segment_and_preprocess(img) for img in tqdm(X_train)])
X_val_processed = np.array([segment_and_preprocess(img) for img in tqdm(X_val)])
X_test_processed = np.array([segment_and_preprocess(img) for img in tqdm(X_test)])

print("Shape of processed X_train:", X_train_processed.shape)
print("Shape of processed X_val:", X_val_processed.shape)
print("Shape of processed X_test:", X_test_processed.shape)

#############################################
# Data Augmentation and Classification Setup
#############################################

augmentor = ImageDataGenerator(
    horizontal_flip=True,
    zoom_range=0.1,
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1
)

train_gen = augmentor.flow(X_train_processed, y_train, batch_size=32)
val_gen = tf.data.Dataset.from_tensor_slices((X_val_processed, y_val)).batch(32).prefetch(tf.data.AUTOTUNE)

# Define ViT Model from timm library
vit_model = timm.create_model('vit_base_patch16_224', pretrained=True)

# Add classification head on top of ViT
x = vit_model.get_classifier()(vit_model.output)
x = tf.keras.layers.GlobalAveragePooling1D()(x)  # Optional if you want to pool the features before classification
x = tf.keras.layers.Dropout(0.3)(x)
out = tf.keras.layers.Dense(1, activation='sigmoid')(x)  # Binary classification output
model = tf.keras.Model(inputs=vit_model.input, outputs=out)

# Compile model
model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-4),
    loss='binary_crossentropy',
    metrics=['accuracy', tf.keras.metrics.AUC(name='auc')]
)

# Early stopping and learning rate scheduling
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_auc', patience=5, mode='max', restore_best_weights=True)
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_auc', patience=2, factor=0.2, mode='max', min_lr=1e-6)

# Train only classification head first
model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=10,
    callbacks=[early_stop, reduce_lr]
)

# Unfreeze and fine-tune the full model
vit_model.trainable = True
model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-5),
    loss='binary_crossentropy',
    metrics=['accuracy', tf.keras.metrics.AUC(name='auc')]
)

model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=20,
    callbacks=[early_stop, reduce_lr]
)

model.save('vit_tb_classifier_segmented.keras')

# Evaluate and visualize on test set
y_pred_probs = model.predict(X_test_processed)
y_pred = (y_pred_probs > 0.5).astype(int)

print("\nClassification Report:")
print(classification_report(y_test, y_pred))
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))

roc_auc = roc_auc_score(y_test, y_pred_probs)
fpr, tpr, _ = roc_curve(y_test, y_pred_probs)
plt.figure(figsize=(6, 4))
plt.plot(fpr, tpr, color='blue', lw=2)
plt.plot([0, 1], [0, 1], color='gray', lw=2, linestyle='--')
plt.title(f"ROC Curve (AUC = {roc_auc:.3f})")
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.show()


In [1]:
model.train()
imgs, labels = next(iter(train_loader))
imgs = imgs.to(device)
labels = labels.float().unsqueeze(1).to(device)

for step in range(100):
    optimizer.zero_grad()
    outputs = model(imgs).squeeze()
    loss = criterion(outputs, labels.view(-1))
    loss.backward()
    optimizer.step()
    print(f"Step {step+1} - Loss: {loss.item():.6f}")


NameError: name 'model' is not defined

In [ ]:
segment_model.summary()